In [9]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
df = pd.read_csv("data/all_data.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HS", "AS", "HST", "AST", 
                    "B365H", "B365D", "B365A",
                    "IWH", "IWD", "IWA",
                    "WHH", "WHD", "WHA"]
df = df[features_to_keep]

In [12]:
name_conversions = {
'St Etienne': 'Saint-Etienne',
'Ajaccio GFCO': 'Ajaccio', 
'Ath Madrid': 'Atletico',
'Ein Frankfurt': 'Frankfurt',
'Espanol': 'Espanyol',
'La Coruna': 'Depor',
'Nurnberg': 'Nuernberg',
'M\'gladbach': 'Gladbach',
'Bayern Munich': 'Bayern',
'Greuther Furth': 'Fuerth',
'Sp Gijon': 'Gijon',
'FC Koln': 'Koeln',
'Ath Bilbao': 'Bilbao',
'Hansa Rostock': 'Rostock',
'Fortuna Dusseldorf': 'Duesseldorf',
'Schalke 04': 'Schalke',
'Werder Bremen': 'Werder',
'Vallecano': 'RayoVallecano'
}

In [13]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))


In [14]:

club_elo_base_url = "http://api.clubelo.com/"
name_mismatches = []
exceptions = []
for team_name in all_teams:
    try:
        if exists("elo/" + team_name + ".csv"):
            continue
        team_name_url = name_conversions.get(team_name, team_name).replace(" ", "")
        team_data = requests.get(club_elo_base_url + team_name_url, stream=True)
        byte_data = team_data.content
        if len(byte_data) == 38:
            # means that only header was returned, so team was not found
            name_mismatches.append(team_name)
            continue
        with open("elo/" + team_name + ".csv", mode='wb') as f:
            f.write(team_data.content)
    except:
        exceptions.append(team_name)
        continue
print("Name mismatches: ", name_mismatches)
print("Exceptions: ", exceptions)

Name mismatches:  ['Kaiserslautern', 'Gimnastic', 'Evian Thonon Gaillard', 'Munich 1860', 'Arles']
Exceptions:  []


In [15]:
import datetime
from functools import partial

# for team in all_teams:
#     df.loc[(df["HomeTeam"] == team) | (df["AwayTeam"] == team)]
# # df = df.apply(add_elo_feature, axis=1)
# df.to_csv("data/all_data_with_elo.csv")
# df

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
        
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], dayfirst=True)
    try:
        elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("data/all_data_with_elo.csv")
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HS,AS,HST,...,B365D,B365A,IWH,IWD,IWA,WHH,WHD,WHA,HomeTeamELO,AwayTeamELO
0,F1,2000-07-28,Marseille,Troyes,3.0,1.0,H,NaN,NaN,NaN,...,NaN,NaN,1.45,3.50,5.00,1.45,3.50,6.00,1690.283447,1580.911621
1,F1,2000-07-28,Paris SG,Strasbourg,3.0,1.0,H,NaN,NaN,NaN,...,NaN,NaN,1.35,3.60,6.50,1.40,3.70,6.50,1719.976318,1636.466431
2,F1,2000-07-29,Auxerre,Sedan,0.0,1.0,A,NaN,NaN,NaN,...,NaN,NaN,1.70,3.10,3.80,1.65,3.25,4.70,1635.098511,1634.386719
3,F1,2000-07-29,Bordeaux,Metz,1.0,1.0,D,NaN,NaN,NaN,...,NaN,NaN,1.55,3.30,4.50,1.50,3.40,6.00,1729.042725,1677.067505
4,F1,2000-07-29,Guingamp,St Etienne,2.0,2.0,D,NaN,NaN,NaN,...,NaN,NaN,2.20,2.80,2.80,2.40,2.90,2.75,1576.033813,1621.786011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37685,F1,2021-12-05,St Etienne,Rennes,0.0,5.0,A,12.0,18.0,2.0,...,3.75,1.85,4.10,3.75,1.85,4.20,3.60,1.80,1551.744507,1709.489502
37686,D1,2021-12-05,Stuttgart,Hertha,2.0,2.0,D,10.0,15.0,3.0,...,3.40,3.30,2.25,3.30,3.25,2.25,3.25,3.25,1623.838501,1620.785278
37687,E0,2021-12-05,Tottenham,Norwich,3.0,0.0,H,17.0,10.0,7.0,...,5.00,8.50,1.43,4.70,7.50,1.40,4.40,8.50,1780.980591,1638.818604
37688,SP1,2021-12-05,Vallecano,Espanol,1.0,0.0,H,7.0,10.0,0.0,...,3.10,3.80,2.15,3.20,3.60,2.15,3.10,3.70,1642.253540,1655.962036
